### Creating a monitoring framework

In [13]:
import os
import glob
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from plotly.graph_objs import Scatter, Bar, Figure, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
%matplotlib inline

In [42]:
df = pd.DataFrame()
for i in glob.glob("*.csv"):
    tmp = pd.read_csv(i)
    df = df.append(tmp)

In [53]:
df['year'] = '2018'
df['mths'] = [int(i.split("-")[1]) for i in df['d_filter']]
df.loc[df['mths'] >= 10, 'year'] = '2017'
df['date'] = df['d_filter'] + '-' + df['year']
df = df[[ u'date', u'home', u'away', u'location', u'home_w', u'phome_w', u'pts_ta', u'pts_tb', u'predictions', u'p_score']]
df['count'] = 1

### Score by Per Day 

In [67]:
percent = df.groupby(["date"])['p_score', 'count'].sum().reset_index()
percent['percent'] = percent['p_score'] / percent['count'].astype(float)
percent['date'] = pd.to_datetime(percent['date'], format="%d-%m-%Y")
percent = percent.sort_values("date").reset_index(drop=True)

### Agg score 

In [70]:
agg_percent = percent[['p_score', 'count']].cumsum()
agg_percent['a_percent'] = agg_percent['p_score'] / agg_percent['count'].astype(float)
agg_percent['date'] = pd.to_datetime(percent['date'], format="%d-%m-%Y")
agg_percent = agg_percent.sort_values("date").reset_index(drop=True)

### Monitoring results 

In [72]:
# Create a trace
time_series = [str(i) for i in percent['date'].tolist()]

daily_ratio = Scatter(x = time_series, y = [round(i, 3) for i in percent['percent'].tolist()], name = "Daily Ratio", yaxis='y1')
agg_ratio = Scatter(x = time_series, y = [round(i, 3) for i in agg_percent['a_percent'].tolist()], name = "Aggregrated Ratio", yaxis='y1')

games_played = Bar(x = time_series, y = percent['count'].tolist(), 
                   name = "Games Played", yaxis='y2', opacity=0.35)

layout = dict(title = 'NBA Prediction Performance', legend=dict(orientation='h', x=.05, y=1),
              yaxis1=dict(title='Model Accuracy (%)', side='left', range=[0.0,1.1]),
              yaxis2=dict(title='Games Played', overlaying='y', side='right', range=[0,50]), 
              width=1100, height=600,
              xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
             )
data = [daily_ratio, agg_ratio, games_played]

fig = dict(data=data, layout=layout)
iplot(fig)

### Upload the prediction files to Dropbox 

In [8]:
base_folder = os.path.abspath(os.getcwd())
os.chdir("..")

In [9]:
from dropbox_functions import authenticate_into_dropbox
from dropbox_functions import load_file_into_my_dropbox, download_file_from_dropbox
account = authenticate_into_dropbox()

In [10]:
def upload_dropbx_data(account, files_to_shift):
    for i in files_to_shift:
        load_file_into_my_dropbox(account, i, folder="nba_games/predictions")
    print("Upload {}".format(i))

In [11]:
os.chdir(base_folder)

In [12]:
for i in glob.glob("*.csv"):
    load_file_into_my_dropbox(account, i)
    print("Upload {}".format(i))

Upload 2017-10-23_predictions.csv
Upload 2017-10-24_predictions.csv
Upload 2017-10-25_predictions.csv
Upload 2017-10-26_predictions.csv
Upload 2017-10-27_predictions.csv
Upload 2017-10-28_predictions.csv
Upload 2017-10-29_predictions.csv
Upload 2017-10-30_predictions.csv
Upload 2017-10-31_predictions.csv
Upload 2017-11-01_predictions.csv
Upload 2017-11-02_predictions.csv
Upload 2017-11-03_predictions.csv
Upload 2017-11-04_predictions.csv
Upload 2017-11-05_predictions.csv
Upload 2017-11-06_predictions.csv
Upload 2017-11-07_predictions.csv
Upload 2017-11-08_predictions.csv
Upload 2017-11-09_predictions.csv
Upload 2017-11-10_predictions.csv
Upload 2017-11-11_predictions.csv
Upload 2017-11-12_predictions.csv
Upload 2017-11-13_predictions.csv
Upload 2017-11-14_predictions.csv
Upload 2017-11-15_predictions.csv
Upload 2017-11-16_predictions.csv
Upload 2017-11-17_predictions.csv
Upload 2017-11-18_predictions.csv
Upload 2017-11-19_predictions.csv
Upload 2017-11-20_predictions.csv
Upload 2017-11